In [ ]:
# [실험 1] 모델 및 하이퍼파라미터 설정
import torch
import torch.nn as nn
import torch.optim as optim

class MLP(nn.Module):
    def __init__(self, input_size=784, hidden_size=100, num_classes=10):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes)
        )
    def forward(self, x):
        return self.layers(x)

# 하이퍼파라미터 설정
learning_rate_exp1 = 1e-4
nb_epochs_exp1 = 3

# 모델, 손실함수, 최적화기 초기화
model_exp1 = MLP().to(device)
criterion_exp1 = nn.CrossEntropyLoss()
optimizer_exp1 = optim.Adam(model_exp1.parameters(), lr=learning_rate_exp1)

# 훈련 과정 추적 리스트 초기화
train_losses_exp1 = []
train_accuracies_exp1 = []
test_accuracies_exp1 = []

print(f"\n=== [실험 1] 모델 (LR={learning_rate_exp1}) 훈련 시작 ===")

# ====================================================================
# [실험 1] 훈련 루프 실행 (LR=1e-4)
# ====================================================================

for epoch in range(nb_epochs_exp1):
    model_exp1.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for batch_idx, batch in enumerate(train_loader):
        imgs = batch["image"].to(device)
        labels = batch["label"].to(device)
        optimizer_exp1.zero_grad()
        outputs = model_exp1(imgs)
        loss = criterion_exp1(outputs, labels)
        loss.backward()
        optimizer_exp1.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
        if (batch_idx + 1) % 100 == 0:
            current_loss = running_loss / (batch_idx + 1)
            current_acc = 100 * correct_train / total_train
            print(f"Epoch [{epoch+1}/{nb_epochs_exp1}], Batch [{batch_idx+1}/{len(train_loader)}]")
            print(f"  Loss: {current_loss:.4f}, Train Acc: {current_acc:.2f}%")
    epoch_loss = running_loss / len(train_loader)
    epoch_train_acc = 100 * correct_train / total_train
    train_losses_exp1.append(epoch_loss)
    train_accuracies_exp1.append(epoch_train_acc)
    print(f"\nEpoch [{epoch+1}/{nb_epochs_exp1}] 훈련 완료:")
    print(f"  평균 Loss: {epoch_loss:.4f}")
    print(f"  훈련 정확도: {epoch_train_acc:.2f}%")
    model_exp1.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for batch in test_loader:
            imgs = batch["image"].to(device)
            labels = batch["label"].to(device)
            outputs = model_exp1(imgs)
            _, predicted = torch.max(outputs, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
    test_acc = 100 * correct_test / total_test
    test_accuracies_exp1.append(test_acc)
    print(f"  테스트 정확도: {test_acc:.2f}%")
    print("-" * 60)
print(f"\n=== 실험 1 훈련 완료 (LR=1e-4) ===")
print(f"최종 테스트 정확도: {test_accuracies_exp1[-1]:.2f}%")